In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, LSTM, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121

# Set the path to your dataset
train_data_dir = r"D:\P_126\Dataset_new\train-20230326T152931Z-001\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')  # Use 'sparse' for integer labels

# Define the DenseNet model for feature extraction
cnn_input = Input(shape=(img_width, img_height, 3))
densenet_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=cnn_input)

# Freeze the layers
for layer in densenet_model.layers:
    layer.trainable = False

# Get the output of the DenseNet model
densenet_output = densenet_model.output

# Flatten the output
flatten_layer = Flatten()(densenet_output)

# Define the LSTM model for sequence modeling
lstm_model = Sequential([
    LSTM(128, input_shape=(1, flatten_layer.shape[1])),
    Dense(len(class_names), activation='softmax')
])
lstm_output = lstm_model(tf.expand_dims(flatten_layer, axis=1))

# Combine the DenseNet and LSTM models
combined_model = Model(inputs=cnn_input, outputs=lstm_output)

# Compile and train the model
combined_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
combined_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50)

# Save the trained model
#combined_model.save('lung_detection_densenet.h5')



Found 9971 images belonging to 6 classes.



Epoch 1/50


312/312 [==============================] - 551s 2s/step - loss: 0.9441 - accuracy: 0.7107
Epoch 2/50
312/312 [==============================] - 482s 2s/step - loss: 0.6558 - accuracy: 0.8022
Epoch 3/50
312/312 [==============================] - 482s 2s/step - loss: 0.6015 - accuracy: 0.8144
Epoch 4/50
312/312 [==============================] - 478s 2s/step - loss: 0.5596 - accuracy: 0.8193
Epoch 5/50
312/312 [==============================] - 479s 2s/step - loss: 0.5278 - accuracy: 0.8287
Epoch 6/50
312/312 [==============================] - 479s 2s/step - loss: 0.5136 - accuracy: 0.8284
Epoch 7/50
312/312 [==============================] - 480s 2s/step - loss: 0.4746 - accuracy: 0.8414
Epoch 8/50
312/312 [==============================] - 479s 2s/step - loss: 0.4862 - accuracy: 0.8369
Epoch 9/50
312/312 [==============================] - 477s 2s/step - loss: 0.4868 - accuracy: 0.8391
Epoch 10/50
312/312 [======================

In [1]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the saved model
model_path = 'lung_detection_dblstm.h5'
loaded_model = load_model(model_path)

# Define your custom class names
class_names = ['CARDIOMEGALY', 'COVID', 'NORMAL', 'PNEUMONIA', 'PNEUMOTHORAX', 'TUBERCULOSIS']

# Path to the image you want to predict
image_path = r"C:\Users\asuto\Desktop\intern ship 2\lung detection\archive\test-20230326T155708Z-001\test\COVID\COVID-19 (8).jpeg"

# Load and preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
img_array = np.expand_dims(img_array, axis=1)  # Add a time dimension
img_array /= 255.0  # Rescale pixel values to [0, 1]

# Make a prediction
predictions = loaded_model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get the predicted class name from the custom class names list
predicted_class_name = class_names[predicted_class]

# Display the predicted class
print(f"Predicted class: {predicted_class_name}")

KeyboardInterrupt: 